In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import config
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,2021-09-25 01:42:01,42.6001,-73.9662,59.63,82,2,2.28
1,1,Wasilla,US,2021-09-25 01:45:09,61.5814,-149.4394,38.03,92,90,0.00
2,2,Ushuaia,AR,2021-09-25 01:41:59,-54.8000,-68.3000,38.86,87,75,0.00
3,3,Esperance,AU,2021-09-25 01:45:10,-33.8667,121.9000,55.24,53,46,14.36
4,4,Hermanus,ZA,2021-09-25 01:43:29,-34.4187,19.2345,53.91,83,100,3.02


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
9,9,Airai,TL,2021-09-25 01:42:41,-8.9266,125.4092,77.65,36,25,2.53
12,12,Abnub,EG,2021-09-25 01:45:13,27.2696,31.1511,75.42,49,0,9.37
15,15,Koumac,NC,2021-09-25 01:45:14,-20.5667,164.2833,75.33,60,35,8.16
18,18,Atuona,PF,2021-09-25 01:43:27,-9.8000,-139.0333,77.88,79,100,16.96
28,28,Victoria,HK,2021-09-25 01:40:43,22.2855,114.1577,89.26,75,62,3.00
31,31,Buala,SB,2021-09-25 01:45:18,-8.1450,159.5921,85.59,68,94,7.47
33,33,Tura,IN,2021-09-25 01:44:22,25.5198,90.2201,81.73,78,11,2.73
39,39,Sumenep,ID,2021-09-25 01:45:21,-7.0167,113.8667,85.23,66,89,10.85
44,44,Vaini,TO,2021-09-25 01:41:14,-21.2000,-175.2000,75.36,69,20,14.97
45,45,Butaritari,KI,2021-09-25 01:43:32,3.0707,172.7902,83.73,73,12,10.83


In [8]:
preferred_cities_df.count()

City_ID       176
City          176
Country       176
Date          176
Lat           176
Lng           176
Max Temp      176
Humidity      176
Cloudiness    176
Wind Speed    176
dtype: int64

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Airai,TL,77.65,-8.9266,125.4092,
12,Abnub,EG,75.42,27.2696,31.1511,
15,Koumac,NC,75.33,-20.5667,164.2833,
18,Atuona,PF,77.88,-9.8000,-139.0333,
28,Victoria,HK,89.26,22.2855,114.1577,
31,Buala,SB,85.59,-8.1450,159.5921,
33,Tura,IN,81.73,25.5198,90.2201,
39,Sumenep,ID,85.23,-7.0167,113.8667,
44,Vaini,TO,75.36,-21.2000,-175.2000,
45,Butaritari,KI,83.73,3.0707,172.7902,


In [10]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [11]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Airai,TL,77.65,-8.9266,125.4092,Juvinal Martin
12,Abnub,EG,75.42,27.2696,31.1511,Romany Adly Gad
15,Koumac,NC,75.33,-20.5667,164.2833,Le Monitel
18,Atuona,PF,77.88,-9.8000,-139.0333,Villa Enata
28,Victoria,HK,89.26,22.2855,114.1577,Mini Hotel Central
31,Buala,SB,85.59,-8.1450,159.5921,Maringe Lagoon Lodge
33,Tura,IN,81.73,25.5198,90.2201,Hotel RIKMAN Continental
39,Sumenep,ID,85.23,-7.0167,113.8667,OYO 3271 Homestay Pandian
44,Vaini,TO,75.36,-21.2000,-175.2000,Keleti Beach Resort
45,Butaritari,KI,83.73,3.0707,172.7902,Isles Sunset Lodge


In [14]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))